In [ ]:
# default_exp functional

In [ ]:
#export
from kesscore.imports import *
from functools import wraps

In [ ]:
#export
def _listify(f=None):
    if f==None: return lisitify_decorator
    @wraps(f)
    def _inner(*args, **kwargs): return list(f(*args, **kwargs))
    return _inner    

In [ ]:
#export
lmap,lzip,lfilter = map(_listify,[map,zip,filter])

In [ ]:
test_eq_type(lmap(noop, [1,2,3]), [1,2,3])
test_eq_type(lzip([1,2],[3,4]),[(1,3),(2,4)])
test_eq_type(lfilter(ge(2), [1,2,3,4]), [2,3,4])

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
show_doc(lmap)

<h4 id="map" class="doc_header"><code>map</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>map</code>()

map(func, *iterables) --> map object

Make an iterator that computes the function using arguments from
each of the iterables.  Stops when the shortest iterable is exhausted.

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted index.ipynb.
